### Ordering a Basemap With a Set of Quad IDs

you can get the quad IDs from planet explorer or if you have the basemap, you can find the quad IDs like this:

In [ ]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'

In [ ]:
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
orders_url = 'https://api.planet.com/compute/ops/orders/v2' 
auth = HTTPBasicAuth(API_KEY, '')

    
# Setup the session
session = requests.Session()

# Authenticate
session.auth = (API_KEY, "")

In [ ]:
response = requests.get(orders_url, auth=session.auth)
response

basemap_params = {
    'name__is': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
headers = {'content-type': 'application/json'}

basemap_params = {
    'name__is': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

basemapServiceResponse = requests.get(url=BASEMAP_API_URL, params=basemap_params, auth=auth, headers=headers)

basemaps = basemapServiceResponse.raise_for_status()
if basemapServiceResponse.status_code != 204:
    basemaps = json.loads(basemapServiceResponse.text)
print(basemaps)

In [ ]:
mosaicID = basemaps['mosaics'][0]['id']
mosaicBbox = basemaps['mosaics'][0]['bbox']
mosaicBboxStr = ','.join(map(str, mosaicBbox))

quad_params = {
    'mosaic_id': mosaicID,
    'bbox': mosaicBboxStr,
}

In [ ]:
quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaicID)
quadServiceResponse = requests.get(url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quadServiceResponse.json()
items = quads['items']

In [ ]:
quadIds = []
for i in items:
    quadId = i['id']
    quadIds.append(quadId)

In [ ]:
order_params = {
   "name":"basemap order with quad_ids",
   "source_type":"basemaps",
   "products":[
      {
         "mosaic_name":"point_reyes_rolling_normalized_2020-04-30_mosaic",
         "quad_ids": quadIds
      }
   ],
    "tools": [
      {
         "reproject": {
        "projection": "EPSG:4326",
        "resolution": 3.0 / 111000.0,
        "kernel": "cubic"
      }
      },
      {
      "bandmath": {
        "b1": "b1",
        "b2": "b2",
        "b3": "b3",
        "b4": "arctan(b1)",
        "b5": "(b4-b3)/(b4+b3)",
        "pixel_type": "32R"
      }
      
      }
   
    ],

    
   "delivery":{
      "google_cloud_storage":{
         "bucket":"notebook-practice",
         "credentials":GCP_CREDENTIALS,
         "path_prefix":"basemaps-to-cloud/",
      }
   }
}

In [ ]:
paramRes = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)

paramRes.raise_for_status()
if paramRes.status_code != 204:
    print(paramRes.text)

In [ ]:
def place_order(order_params, auth):
    response = requests.post(ORDERS_API_URL, data=json.dumps(order_params), auth=auth, headers=headers)
    print(response)
    order_id = response.json()['id']
    print(order_id)
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

In [ ]:
order_url = place_order(order_params, session.auth)

In [ ]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
        
poll_for_success(order_url, auth)

In [ ]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

In [ ]:
[r['name'] for r in results]